<div class="alert alert-block alert-info">
<center> <h1> Customer Segmentation and Sales Forecast</h1> </center> <br>
<center> Big Data Analytics 2025</center><br>
<center> NOVA IMS MDSAA</center>

### [NOTE]
In this project, we are going to work on 4 notebooks:
- 1. **Data Preprocessing**: For EDA, Data Preprocessing, Creating DataFrames, and Feature Engineering
- 2. **Clustering**: For Clustering 
- 3. **Sales Forecasting**: For Sales Forecast
- 4. **Graph**: For Graph Visualization for Clusters
<br>
##### This notebook is 3. Sales Forecasting.

# Group 77

|   | Student Name          |  Student ID | 
|---|-----------------------|    ---      |
| 1 | Hassan Bhatti       |  20241023 |
| 2 | Moeko Mitani          |   20240670  | 
| 3 | Oumayma Ben Hfaiedh   |   20240699  | 
| 4 | Ricardo Pereira      |  20240745  | 

# 1. Data Integration 

## 1.1. Import Libraries

In [0]:
# Core PySpark
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import (
    col, lit, to_date, date_format, year, month, dayofmonth,
    sin, cos, avg, stddev, lag, when, countDistinct,
    sum as spark_sum, max as spark_max, monotonically_increasing_id,
    datediff, current_date
)
from pyspark.sql.window import Window

# PySpark ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    VectorAssembler, PCA, StringIndexer, StandardScaler
)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.regression import LinearRegression

# Pandas, NumPy, Matplotlib, Seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm

# Scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, silhouette_score, silhouette_samples, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Date utils
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [0]:
# Start Spark session
spark = SparkSession.builder.appName("Project_Group77").getOrCreate()

## 1.2. Import CSV File (from previous Notebook: 1. Data Preprocessing)

In [0]:
# File location and type
file_location = "/FileStore/df_sf.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_final = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_final.limit(10).display()
df_final.schema


StockCode,month,total_quantity,date,year,month_num,month_sin,month_cos,lag_6m_quantity,rolling_avg_6m,rolling_std_6m
10135,2023-06-01,73.0,2023-06-01,2023,202306,1.2246467991473532E-16,-1.0,123.0,213.83333333333334,321.93130737265466
10135,2023-07-01,53.0,2023-07-01,2023,202307,-0.4999999999999997,-0.8660254037844388,48.0,214.66666666666666,321.42225602261374
10135,2023-08-01,74.0,2023-08-01,2023,202308,-0.8660254037844385,-0.5000000000000004,868.0,82.33333333333333,29.770231216211048
10135,2023-09-01,45.0,2023-09-01,2023,202309,-1.0,-1.8369701987210297E-16,101.0,73.0,31.476975712415573
10135,2023-10-01,84.0,2023-10-01,2023,202310,-0.8660254037844386,0.5000000000000001,60.0,77.0,31.016124838541646
10135,2023-11-01,325.0,2023-11-01,2023,202311,-0.5000000000000004,0.8660254037844384,133.0,109.0,106.80262169066825
10135,2023-12-01,411.0,2023-12-01,2023,202312,-2.4492935982947064E-16,1.0,73.0,165.33333333333334,159.93832144507041
10135,2024-01-01,613.0,2024-01-01,2024,202401,0.49999999999999994,0.8660254037844387,53.0,258.6666666666667,229.53053536875365
10135,2024-02-01,272.0,2024-02-01,2024,202402,0.8660254037844386,0.5000000000000001,74.0,291.6666666666667,211.16975793580545
10135,2024-03-01,118.0,2024-03-01,2024,202403,1.0,6.123233995736766E-17,45.0,303.8333333333333,195.64806839492863


Out[3]: StructType([StructField('StockCode', StringType(), True), StructField('month', DateType(), True), StructField('total_quantity', DoubleType(), True), StructField('date', DateType(), True), StructField('year', IntegerType(), True), StructField('month_num', IntegerType(), True), StructField('month_sin', DoubleType(), True), StructField('month_cos', DoubleType(), True), StructField('lag_6m_quantity', DoubleType(), True), StructField('rolling_avg_6m', DoubleType(), True), StructField('rolling_std_6m', DoubleType(), True)])

# 2. Split DataFrame

We are going to split the data into three time-based sets: train set (*df_train*), validation set (*df_val*), and test set (*df_test*).
- Train set (*df_train*): The data set where the month is between June 2023 and June 2024 (1 year).
- Validation set (*df_val*): The data set with months from July 2024 to December 2024 (6 months).
- Test set (*df_test*): The data set from January 2025 to June 2025 (6 months).

In [0]:
# Train: 2023-06 to 2024-06
df_train = df_final.filter((col("month") >= "2023-06") & (col("month") <= "2024-06"))

# Validation: 2024-07 to 2024-12
df_val = df_final.filter((col("month") >= "2024-07") & (col("month") <= "2024-12"))

# Test: 2025-01 to 2025-06
df_test = df_final.filter((col("month") >= "2025-01") & (col("month") <= "2025-06"))


# 3. Model Assessment
Because of the limitations of databricks communty edition we couldn't do forecasting for all products. That's why we chose to only run predictions for only **one product: 10135**.

## 3.1. Comparison between Different Models

We are going to apply different models; **Linear Regression, Random Forest, Gradient Boost**, and **SARIMAX**, to check which model works best for our dataset.

In [0]:
# Define Features and Target 
features_LR = ['month_sin', 'rolling_avg_6m', 'rolling_std_6m']
target = 'total_quantity'
test_stock_code = '10135'  # Chosen product

# Filter Data and Convert to Pandas
train_product = df_train.filter(col("StockCode") == test_stock_code).dropna().toPandas()
val_product = df_val.filter(col("StockCode") == test_stock_code).dropna().toPandas()
test_product = df_test.filter(col("StockCode") == test_stock_code).toPandas()

# Proceed if Training and Validation Data Exist
if not train_product.empty and not val_product.empty:
    rmse_scores = {}

    # LINEAR REGRESSION
    model_lr = LinearRegression()
    model_lr.fit(train_product[features_LR], train_product[target])
    val_product['predicted_LR'] = model_lr.predict(val_product[features_LR])
    test_product['predicted_LR'] = model_lr.predict(test_product[features_LR])
    rmse_scores['Linear Regression'] = np.sqrt(mean_squared_error(val_product[target], val_product['predicted_LR']))

    # RANDOM FOREST
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(train_product[features_LR], train_product[target])
    val_product['predicted_RF'] = model_rf.predict(val_product[features_LR])
    test_product['predicted_RF'] = model_rf.predict(test_product[features_LR])
    rmse_scores['Random Forest'] = np.sqrt(mean_squared_error(val_product[target], val_product['predicted_RF']))

    # GRADIENT BOOSTING
    model_gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model_gb.fit(train_product[features_LR], train_product[target])
    val_product['predicted_GB'] = model_gb.predict(val_product[features_LR])
    test_product['predicted_GB'] = model_gb.predict(test_product[features_LR])
    rmse_scores['Gradient Boosting'] = np.sqrt(mean_squared_error(val_product[target], val_product['predicted_GB']))

    # SARIMAX
    try:
        train_product['month'] = pd.to_datetime(train_product['month'])
        val_product['month'] = pd.to_datetime(val_product['month'])
        test_product['month'] = pd.to_datetime(test_product['month'])

        train_ts = train_product.set_index('month')[target]
        sarimax_model = SARIMAX(train_ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarimax_fit = sarimax_model.fit(disp=False)

        total_steps = len(val_product) + len(test_product)
        sarimax_preds_all = sarimax_fit.forecast(steps=total_steps)

        val_product['predicted_SARIMAX'] = sarimax_preds_all[:len(val_product)].values
        test_product['predicted_SARIMAX'] = sarimax_preds_all[len(val_product):].values

        rmse_scores['SARIMAX'] = np.sqrt(mean_squared_error(val_product[target], val_product['predicted_SARIMAX']))
    except Exception as e:
        print(f" SARIMAX failed: {e}")
        val_product['predicted_SARIMAX'] = np.nan
        test_product['predicted_SARIMAX'] = np.nan

    # Display RMSE Scores
    print("\n Validation RMSE Comparison:")
    rmse_df = pd.DataFrame.from_dict(rmse_scores, orient='index', columns=['RMSE']).sort_values('RMSE')
    print(rmse_df)

    # Final Forecast Tables
    print("\n Final Forecast (Validation Set):")
    display(val_product[['StockCode', 'month', 'total_quantity',
                         'predicted_LR', 'predicted_RF', 'predicted_GB', 'predicted_SARIMAX']])

    print("\n Final Forecast (Test Set):")
    display(test_product[['StockCode', 'month',
                          'predicted_LR', 'predicted_RF', 'predicted_GB', 'predicted_SARIMAX']])  # SARIMAX not applied to test

else:
    print(f" No data found for StockCode {test_stock_code}")

/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate startin


 Validation RMSE Comparison:
                         RMSE
Random Forest       79.070812
Linear Regression   95.897447
Gradient Boosting  101.888456
SARIMAX            159.108977

 Final Forecast (Validation Set):


StockCode,month,total_quantity,predicted_LR,predicted_RF,predicted_GB,predicted_SARIMAX
10135,2024-07-01T00:00:00.000+0000,177.0,183.53151625594995,257.56,325.0227764766096,35.0
10135,2024-08-01T00:00:00.000+0000,151.0,114.77666426518124,81.32,67.9253201050212,56.0
10135,2024-09-01T00:00:00.000+0000,70.0,94.20955799706707,76.57,59.72889779492801,26.999999999999996
10135,2024-10-01T00:00:00.000+0000,68.0,116.83828828102338,81.32,67.9253201050212,66.0
10135,2024-11-01T00:00:00.000+0000,181.0,157.54040685189557,185.81,229.35836421472166,307.0
10135,2024-12-01T00:00:00.000+0000,69.0,293.27995289765147,230.01,245.15722462458342,393.0



 Final Forecast (Test Set):


StockCode,month,predicted_LR,predicted_RF,predicted_GB,predicted_SARIMAX
10135,2025-01-01T00:00:00.000+0000,459.99787773671176,228.97,244.6165738039354,595.0
10135,2025-02-01T00:00:00.000+0000,588.2413517807257,218.51,222.06269476211256,254.0
10135,2025-03-01T00:00:00.000+0000,608.9785792853827,212.43,222.52622923948397,100.0
10135,2025-04-01T00:00:00.000+0000,533.1979237017906,273.76,336.9330671385778,12.0
10135,2025-05-01T00:00:00.000+0000,553.538932106824,215.08,231.14352390988338,-5.0
10135,2025-06-01T00:00:00.000+0000,411.801672311257,221.05,231.6841747305314,37.0


Because of the lowest RMSE, we are going to choose Random Forest as our model.

## 3.2. Hypertuning Random Forest

To build better and more efficient model, we are going to do hypertuning for our Random Forest model.

In [0]:
# Features and target
features_LR = ['month_sin', 'rolling_avg_6m', 'rolling_std_6m']
target = 'total_quantity'
test_stock_code = '10135'

# Get training/validation data
train_product = df_train.filter(col("StockCode") == test_stock_code).dropna().toPandas()
val_product = df_val.filter(col("StockCode") == test_stock_code).dropna().toPandas()

if not train_product.empty and not val_product.empty:
    X_train = train_product[features_LR]
    y_train = train_product[target]
    X_val = val_product[features_LR]
    y_val = val_product[target]

    # Define hyperparameter grid
    param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2']
    }

    # Set up search
    rf = RandomForestRegressor(random_state=42)
    search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                n_iter=20, cv=3, scoring='neg_root_mean_squared_error',
                                random_state=42, n_jobs=-1, verbose=1)

    # Fit
    search.fit(X_train, y_train)

    # Best model
    best_rf = search.best_estimator_
    print(f"\n Best Parameters: {search.best_params_}")

    # Predict and evaluate
    val_product['predicted_RF_Tuned'] = best_rf.predict(X_val)
    rmse_tuned = np.sqrt(mean_squared_error(y_val, val_product['predicted_RF_Tuned']))
    print(f" Tuned Random Forest RMSE (Validation): {rmse_tuned:.2f}")

    # Output forecast
    display(val_product[['StockCode', 'month', 'total_quantity', 'predicted_RF_Tuned']])

else:
    print(f" No data found for StockCode {test_stock_code}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits

 Best Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20}
 Tuned Random Forest RMSE (Validation): 70.76


StockCode,month,total_quantity,predicted_RF_Tuned
10135,2024-07-01,177.0,172.1565622433122
10135,2024-08-01,151.0,165.20587705812702
10135,2024-09-01,70.0,165.20587705812702
10135,2024-10-01,68.0,165.20587705812702
10135,2024-11-01,181.0,171.45461779886776
10135,2024-12-01,69.0,174.90716770266766


By doing hypertuning, **we improved the RMSE score from 79.07 to 70.76**.